In [14]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import IPython

In [16]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [17]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='euJhZdzTIm8pop9_skQQSAuXh4dmvLIFYuR1N2Mq5NTe',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'music-donotdelete-pr-nklfx6cua62dyo'
object_key = 'my1.dat'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
import os


f=open("my1.dat",'wb')
i=0
#Dataset creation : making a dat file were we get all the data about the audio files in a ".dat" file. 
for folder in os.listdir(directory):
#as we have 10 classes, we're starting the loop from 1 to 11
#so that we run the loop for total of 10 times, with each folder change (genre change), i (label) changes. 
    i += 1
    if i==11 :
        break
    for file in os.listdir(directory+folder):
#To read an Wav audio File in Python
        (rate,sig) = wav.read(directory+folder+"/"+file)
#MFCC is the feature we will use for our analysis,
#because it provides data about the overall shape of the audio frequencies.
        mfcc_feat = mfcc (sig, rate,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean (0)
#making a feature typle that contains the mean matrix from mfcc as well as covariance,
#and last variable in the feature tuple is the label (where numbers correspond to particular genre) 
        feature = (mean_matrix, covariance, i)
#This the the created dataset, which takes the input from specified path
#it then stores the feature as a .dat file which can be used later without having to train all the files over it. 
        pickle.dump(feature, f)
f.close()

In [7]:
#Loading the created dataset into a python readable object (list) dataset = []
dataset=[]
def loadDataset (filename):
    with open("F:/notebook/jeemol/project/Flask/my1.dat", 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError: 
                f.close() 
                break
loadDataset("F:/notebook/jeemol/project/Flask/my1.dat")
#we have to convert the dataset from a list to np array. 
dataset = np.array(dataset)
#type(dataset) ##uncomment this line to check the type of (dataset),

In [8]:
#Train and test split on the dataset:
#as the dataset contains features for all the audio files, #we have to split that manually into train and test data 
from sklearn.model_selection import train_test_split 
x_train,x_test = train_test_split(dataset, test_size=0.15)
#Make prediction using KNN and get the accuracy on test data: 
leng = len(x_test)
predictions = []
for x in range (leng):
    predictions.append(nearestClass (getNeighbors (x_train,x_test[x], 8)))

In [19]:
!pip install -U ibm-watson-machine-learning
!tar -zcvf music.tgz parkinson.pkl

tar: parkinson.pkl: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [20]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com", 
                 "apikey": "TqjVLPC1pWR6M1FhYVyZBxZ-dc5w4GVCdK_d3i8OelAQ"
                }
client=APIClient(wml_credentials)

In [21]:
def guide_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']["name"]==space_name)['metadata']['id'])


In [22]:
space_uid=guide_from_space_name(client,'music')
print("Space UID =" + space_uid)

Space UID =5eced59d-f9a1-454d-8789-1e71623e092e


In [23]:
client.set.default_space(space_uid)

'SUCCESS'

In [24]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----  ------------------  -------------------------------
NAME                           ID                                    TYPE  STATE               REPLACEMENT
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  unsupported         runtime-22.1-py3.9
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base  not_provided
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base  not_provided
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  unsupported         runtime-22.1-py3.9
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  unsupported
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base  supported
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base  unsupported         runtime-22.1-py3.9
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base  n

In [25]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [18]:
model_details=client.repository.store_model(model='music.tgz',meta_props={client.repository.ModelMetaNames.NAME:"parkinson_model",client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_id(model_details)

In [19]:
model_id

'858ef1f1-a8fb-4c67-8af4-b61cfdb1b8d1'